# Modeling Crop Yield Anomaly
## Python modules

In [1]:
## import warnings
import time
import math
import os
import glob
from pyhere import here
from datetime import date
import re

import numpy as np
import pandas as pd
import geopandas

import pyarrow
import itertools
import multiprocessing
import p_tqdm

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split, KFold, LeaveOneGroupOut, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import r2_score
from scipy.stats import spearmanr,  pearsonr

In [2]:
def str2bool(string):
    return string.lower() in ("yes", "true", "t", "1")

point_pattern = re.compile("20k-points")
wa_pattern = re.compile("cm-False")

In [3]:
data_dir = here("data")
directory = here("data", "random_features", "summary")
today = date.today().strftime("%Y-%m-%d")
files = os.listdir(directory)
files = [f for f in files if f not in ('.gitkeep', '.ipynb_checkpoints')]
files = [f for f in files if not (bool(point_pattern.search(f)) & bool(wa_pattern.search(f)))]
len(files)

44

In [4]:
def model(params):
    #########################################     SET PARAMS    #########################################
    file         = params
    f            = file.split(sep="_")
    satellite    = f[0]
    bands        = f[1].replace("bands-", "")
    country_code = f[2]
    points       = f[3].replace("k-points", "")
    num_features = f[4].replace("-features", "")
    yrs          = f[5].replace("yr-", "").split(sep="-")
    mns          = f[6].replace("mn-", "").split(sep="-")
    limit_months = str2bool(f[7].replace("lm-", ""))
    crop_mask    = str2bool(f[8].replace("cm-", ""))
    weighted_avg = str2bool(f[9].replace("wa-", ""))
    years        = range(int(yrs[0]), int(yrs[1])+1)
    month_range  = list(range(int(mns[0]), int(mns[1])+1))
    
    #########################################     READ DATA    #########################################
    fn = f"{directory}/{file}"
    features = pd.read_feather(fn)
     
    drop_cols = ['district', 'year', 'yield_mt', "crop_perc"]
            
    if weighted_avg:
        drop_cols.remove("crop_perc")
    else:
        pass

    #########################################     CALCULATE ANOMALY   #########################################
    features['yield_mt'] = np.log10(features['yield_mt'] + 1)
    features.set_index(['year', 'district'], inplace=True)
    var_cols = features.columns
    features = features[var_cols] - features.groupby(['district'], as_index=True)[var_cols].transform('mean')
    features.reset_index(drop=False, inplace=True)
    
    #########################################     SPLIT DATA    #########################################
    x_all = features.drop(drop_cols, axis = 1) 
    y_all = features.yield_mt
    g_all = features.year.ravel()
    
    x_train, x_test,\
    y_train, y_test,\
    g_train, g_test = train_test_split(x_all, y_all, g_all, test_size = 0.2, random_state = 0)

    #########################################     K-FOLD CV    ###########################################
    kfold = KFold()
    ridge = Ridge()
    parameters = {'alpha': np.logspace(-8, 8, base = 10, num = 17)}
    ### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
    ridge_kfold_reg = GridSearchCV(ridge, parameters, scoring = 'r2', cv = kfold)
    ridge_kfold_reg.fit(x_train, y_train)
    best_kfold_model = ridge_kfold_reg.best_estimator_
    ### CV PREDICT - PREDICTING WITH BEST HYPERPARAMETER
    kfold_val_predictions = cross_val_predict(best_kfold_model, X = x_train, y = y_train, cv = kfold)   
    ### TRAIN BEST MODEL AND PREDICT
    best_kfold_model.fit(x_train, y_train)
    y_pred_train = np.maximum(best_kfold_model.predict(x_train), 0)
    y_pred_test  = np.maximum(best_kfold_model.predict(x_test), 0)

    #########################################     LOGO CV    ###########################################
    logo = LeaveOneGroupOut()
    ridge = Ridge()
    parameters = {'alpha': np.logspace(-8, 8, base = 10, num = 17)}
    ### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
    ridge_logo_reg = GridSearchCV(ridge, parameters, scoring = 'r2', cv = logo)
    ridge_logo_reg.fit(x_all, y_all, groups = g_all)
    best_logo_model = ridge_logo_reg.best_estimator_
    ### CV PREDICT - PREDICTING WITH BEST HYPERPARAMETER
    logo_val_predictions = cross_val_predict(best_logo_model, X = x_all, y = y_all, groups = g_all,  cv = logo)   
    
    #########################################     SAVE RESULTS    #########################################
    d = {
        'country': country_code,
        'satellite': satellite,
        'bands': bands,
        'num_features': num_features,
        'points': points, 
        'month_range': f'{min(month_range)}-{max(month_range)}',
        
        'limit_months': limit_months,
        'crop_mask': crop_mask,
        'weighted_avg': weighted_avg,
        
        'total_n': len(x_all),
        'train_n': len(x_train),
        'test_n': len(x_test),
        
        'kfold_best_reg_param': list(ridge_kfold_reg.best_params_.values())[0],
        'kfold_mean_of_val_R2s': ridge_kfold_reg.best_score_,
        'kfold_val_R2': r2_score(y_train, kfold_val_predictions),
        'kfold_val_r' : pearsonr(kfold_val_predictions, y_train)[0],
        'kfold_val_r2' : pearsonr(kfold_val_predictions, y_train)[0] ** 2,
        
        'kfold_train_R2': r2_score(y_train, y_pred_train),
        'kfold_train_r': pearsonr(y_pred_train, y_train)[0],
        'kfold_train_r2': pearsonr(y_pred_train, y_train)[0] ** 2,
        
        'kfold_test_R2': r2_score(y_test, y_pred_test),
        'kfold_test_r': pearsonr(y_pred_test, y_test)[0],
        'kfold_test_r2': pearsonr(y_pred_test, y_test)[0] ** 2,
                
        'logo_best_reg_param': list(ridge_logo_reg.best_params_.values())[0],      
        'logo_mean_of_val_R2s' : ridge_logo_reg.best_score_,
        'logo_val_R2' : r2_score(y_all, logo_val_predictions),
        'logo_val_r' : pearsonr(logo_val_predictions, y_all)[0],
        'logo_val_r2' : pearsonr(logo_val_predictions, y_all)[0] ** 2
    }
    return pd.DataFrame(data=d, index=[0])

In [5]:
%%time     
##### With progress bar
workers = 44 ## os.cpu_count()
if __name__ == "__main__":
    output = []
    for result in p_tqdm.p_map(model, files, num_cpus=workers):
        output.append(result)
    results = pd.concat(output).reset_index(drop=True)
    today = date.today().strftime("%Y-%m-%d")
    file_name = f'anomaly_results_{today}.csv'
    print(f"Saving results as: {file_name}\n\n")           
    results.to_csv(here("data","results", file_name))

  0%|          | 0/44 [00:00<?, ?it/s]

Saving results as: anomaly_results_2022-11-14.csv


CPU times: user 575 ms, sys: 449 ms, total: 1.02 s
Wall time: 16min 56s


In [6]:
results.iloc[:, 14:]

,kfold_val_R2,kfold_val_r,kfold_val_r2,kfold_train_R2,kfold_train_r,kfold_train_r2,kfold_test_R2,kfold_test_r,kfold_test_r2,logo_best_reg_param,logo_mean_of_val_R2s,logo_val_R2,logo_val_r,logo_val_r2
0,0.228561,0.491287,0.241363,0.148435,0.458271,0.210012,0.013648,0.355144,0.126127,1.000000e-05,0.206067,0.490617,0.716921,0.513976
1,0.269965,0.550311,0.302843,0.219811,0.580455,0.336928,0.017604,0.319307,0.101957,1.000000e-02,0.344305,0.628017,0.797162,0.635467
2,0.230844,0.493565,0.243606,0.150149,0.460449,0.212013,0.003204,0.355937,0.126691,1.000000e-05,0.218348,0.496158,0.719066,0.517056
3,0.214047,0.490510,0.240600,0.133893,0.440619,0.194145,0.037733,0.263032,0.069186,1.000000e-08,0.658009,0.765443,0.875799,0.767024
4,0.440699,0.669150,0.447761,0.258403,0.621533,0.386303,-0.102182,0.322561,0.104046,1.000000e-05,0.151159,0.487494,0.736424,0.542320
5,0.311502,0.560288,0.313923,0.117965,0.425699,0.181219,0.039055,0.304628,0.092798,1.000000e+08,-0.438929,-0.109024,-0.549270,0.301698
6,0.223160,0.474583,0.225229,0.083661,0.352607,0.124332,-0.051069,0.048097,0.002313,1.000000e-08,0.403392,0.605435,0.788334,0.621470
7,0.217993,0.505632,0.255664,0.196804,0.529840,0.280730,0.060819,0.365841,0.133839,1.000000e-08,0.358784,0.661460,0.826341,0.682839
8,0.277239,0.565803,0.320133,0.260233,0.610548,0.372769,0.102550,0.461180,0.212687,1.000000e-08,-0.182159,0.254439,0.733491,0.538009
9,0.441359,0.669806,0.448641,0.262441,0.612766,0.375483,0.075631,0.439793,0.193418,1.000000e+08,-0.435667,-0.107246,-0.545863,0.297966


In [7]:
# %%time    
##### No progress bar
# if __name__ == "__main__":
#     with multiprocessing.Pool(processes=os.cpu_count()) as pool:
#         output = []
#         for result in pool.imap_unordered(model, paramlist, chunksize=2):
#             output.append(result)
#     results = pd.concat(output).reset_index(drop=True)
#     file_name = f'results_{today}.csv'
#     print(f"Saving results as: {file_name}\n\n")           
#     results.to_csv(here("data","results", file_name))